# Mediation Analysis with Partial Correlation

### Part 1
Various functions for naive mediation analysis with partial correlation, including a weigthed variation.
Partial correlation is calculated as follows:

$ \hat{\rho}_{XY \dot{Z}} = \frac{N\sum_{i=1}^N e_{X,i}e_{Y,i}}{\sqrt{N\sum_{i=1}^N e_{X,i}^2}\sqrt{N\sum_{i=1}^N e_{Y,i}^2}}$

Where $e_{X,i}, e_{Y,i}$ are the residuals.

Here samples are analysed with respect to the given weight.

In [1]:
import pandas as pd
import numpy as np
from patsy import dmatrices
from scipy.stats import pearsonr
from scipy.special import betainc 
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sl
from sklearn import linear_model, datasets, preprocessing, metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold, cross_validate

In [2]:
# P value calc for pearson test

def my_pvalue(r, n):
    df = n - 2
    t_squared = r**2 * (df / ((1.0 - r) * (1.0 + r)))
    x = df / (df + t_squared)
    x = np.array(x)
    x = np.where(x < 1.0, x, 1.0)

    return betainc(0.5*df, 0.5, x)

In [3]:
import sys
from math import sqrt

def wpearson(vec_1, vec_2, weights, r = 4):

	list_length = len(vec_1)

	try:
		weights = list(map(float,weights))
	except:
		print('Invalid weights.')
	
	try:
		vec_1 = list(map(float,vec_1))
		vec_2 = list(map(float,vec_2))
		if any(len(x) != list_length for x in [vec_2,weights]):
			print('Vector/Weight sizes not equal.')
	except Exception as e:
		print('Invalid vectors. \n{}'.format(e.with_traceback))


# Find total weight sum.

	w_sum = sum(weights)

# Calculate the weighted average relative value of vector 1 and vector 2.
	vec1_sum = 0.0
	vec2_sum = 0.0

	for x in range(len(vec_1)):
		vec1_sum += (weights[x] * vec_1[x])
		vec2_sum += (weights[x] * vec_2[x])
	vec1_avg = (vec1_sum / w_sum)
	vec2_avg = (vec2_sum / w_sum)

# Calculate wPCC

	sum_top = 0.0
	sum_bottom1 = 0.0
	sum_bottom2 = 0.0

	for x in range(len(vec_1)):
		dif_1 = (vec_1[x] - vec1_avg)
		dif_2 = (vec_2[x] - vec2_avg)
		sum_top += (weights[x] * dif_1 * dif_2)
		sum_bottom1 += (dif_1**2)*(weights[x])
		sum_bottom2 += (dif_2**2)*(weights[x])

	cor = sum_top / (sqrt(sum_bottom1 * sum_bottom2))

	return round(cor,r)

In [28]:
# For a weighted calculation t
weights = np.load('weights.npy', mmap_mode='r')

In [37]:
from scipy import stats, linalg

def partial_corr(C, W = None):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    weight : weights for calculating weighted correlation
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """

    C = np.asarray(C)
    
    W = np.ones(C.shape[0]) if W is None else W
    
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    P_value = np.zeros((p, p), dtype=np.float)

    for i in range(p):
        P_corr[i, i] = 1
        P_value[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            
            #Calcultae residuals given weights
            A_j = C[:, j]
            A_i = C[:, i]
            B = C[:, idx]
            
            #Find betas with linear regression
            reg_j = LinearRegression(fit_intercept=False)
            reg_j.fit(A_j.reshape(-1, 1),B, sample_weight=W )
            reg_j.fit(B,A_j)

            beta_i = reg_j.coef_
            
            reg_i = LinearRegression(fit_intercept=False)
            reg_i.fit(A_i.reshape(-1,1),B,W )
            reg_i.fit(B,A_i)
            beta_j = reg_i.coef_
  
            
            beta_i = linalg.lstsq(B, A_j)[0]
            beta_j = linalg.lstsq(B, A_i)[0]

# #             print(B.dot(beta_i).reshape(1,-1))
# #             print(A_i)
            res_j = A_j - B.dot(beta_i)
            res_i = A_i - B.dot(beta_j)

            corr = wpearson(res_i, res_j, W)

            P_corr[i, j] = corr
            P_corr[j, i] = corr
            
#             p_value = stats.pearsonr(res_i, res_j)[1]
            p_value = my_pvalue(corr, SAMPLE_SIZE)
            P_value[i, j] = p_value
            P_value[j, i] = p_value

    return P_corr,P_value

In [21]:
More = pd.read_csv('More.csv')
More = More.astype(float)
SAMPLE_SIZE = More.shape[0]

# mini test
Mini = More[['maud5','eatd_py','any_manic_py','any_anx_py','any_disphor_py','eae_violence_1']]
partial_corr_array = Mini

In [14]:
# Some utils

# Check len in pxl
def get_width(num_chars):
    return int((1 + num_chars) *256)

# PC mat to preaty
def allVSall_to_df(mat, trio):
    mat = np.delete(mat, 0, 0)
    mat = np.delete(mat, 0, 1)
    
    df = pd.DataFrame(mat)
    df.columns = trio
    df['Var'] = trio
    df = df.set_index('Var')
    
    return df


In [38]:
pair = partial_corr(partial_corr_array)

conditianl_cov_for_all = pair[0]
conditianl_pvalue_for_all = pair[1]
conditianl_cov_for_all

array([[ 1.    ,  0.5278,  0.5771,  0.0489],
       [ 0.5278,  1.    , -0.0785,  0.0711],
       [ 0.5771, -0.0785,  1.    , -0.0276],
       [ 0.0489,  0.0711, -0.0276,  1.    ]])

### Part 2
Trio analysis - scaling up inputs. Analysing multiple trio, conditioning on different var at a time as suspected mediator.
Output: Trio's Excel which marks significant decrease in correlation after conditioning. 
**Note: Make sure initial correlation is large enough.**

In [25]:
M_s = ['cann_init_age_1', 'nico_init_age_1','alc_init_age_1','not_alc_nico_cann_init_age_1',
      'ppyaud5','pnicdep5','any_anx_py','any_manic_py','eatd_py',
       'personality_dis2','any_disphor_py','any_disphor_ppy','isel_app_1', 'isel_app_2',
       'isel_app_3', 'isel_app_4', 'isel_app_5', 'isel_bel_1', 'isel_bel_2',
       'isel_bel_3', 'isel_bel_4', 'isel_bel_5', 'isel_tan_1', 'isel_tan_2',
       'isel_tan_3', 'isel_tan_4', 'isel_tan_5']


# Customize trios
# CheckList1 =[['personality_dis2','nico_init_age_1','maud5'],['alc_init_age_1','pnicdep5','maud5'],
#              ['eae_sexual','alc_init_age_1','pyaud5'],['alc_init_age_1','eae_violence_1','pyaud5'],
#             ['alc_init_age_1','eae_sexual','any_sud_py'],['alc_init_age_1','eae_violence_1','any_sud_py']]

# comibinations
CheckList1 = []
EAE_event = 'All_EAE' # change accordingly: 'eae_violence_1'/ ''eae_neglect_1' etc.

for m in M_s:
    tr = [EAE_event, m,'maud5']
    CheckList1.append(tr)

print(CheckList1)
# More.corr()['n14q7_4']['anysud_maud']

[['All_EAE', 'cann_init_age_1', 'maud5'], ['All_EAE', 'nico_init_age_1', 'maud5'], ['All_EAE', 'alc_init_age_1', 'maud5'], ['All_EAE', 'not_alc_nico_cann_init_age_1', 'maud5'], ['All_EAE', 'ppyaud5', 'maud5'], ['All_EAE', 'pnicdep5', 'maud5'], ['All_EAE', 'any_anx_py', 'maud5'], ['All_EAE', 'any_manic_py', 'maud5'], ['All_EAE', 'eatd_py', 'maud5'], ['All_EAE', 'personality_dis2', 'maud5'], ['All_EAE', 'any_disphor_py', 'maud5'], ['All_EAE', 'any_disphor_ppy', 'maud5'], ['All_EAE', 'isel_app_1', 'maud5'], ['All_EAE', 'isel_app_2', 'maud5'], ['All_EAE', 'isel_app_3', 'maud5'], ['All_EAE', 'isel_app_4', 'maud5'], ['All_EAE', 'isel_app_5', 'maud5'], ['All_EAE', 'isel_bel_1', 'maud5'], ['All_EAE', 'isel_bel_2', 'maud5'], ['All_EAE', 'isel_bel_3', 'maud5'], ['All_EAE', 'isel_bel_4', 'maud5'], ['All_EAE', 'isel_bel_5', 'maud5'], ['All_EAE', 'isel_tan_1', 'maud5'], ['All_EAE', 'isel_tan_2', 'maud5'], ['All_EAE', 'isel_tan_3', 'maud5'], ['All_EAE', 'isel_tan_4', 'maud5'], ['All_EAE', 'isel_tan_

In [26]:
import xlwt
from copy import deepcopy

# Patterns
# headers
header_pattern = xlwt.Pattern()
header_pattern.pattern = xlwt.Pattern.SOLID_PATTERN
header_pattern.pattern_fore_colour = 30
header_font = xlwt.Font()
header_font.bold = True
header_font.colour_index = 9
align = xlwt.Alignment()
align.horz = xlwt.Alignment.HORZ_CENTER
thick_border = xlwt.Borders()
thick_border.right = xlwt.Borders.THICK
thick_border.left = xlwt.Borders.THICK
thick_border.top = xlwt.Borders.THICK
thick_border.bottom = xlwt.Borders.THICK

header_style = xlwt.XFStyle()
header_style.pattern = header_pattern
header_style.borders = thick_border
header_style.font = header_font

# normal cell

cell_pattern = xlwt.Pattern()
cell_pattern.pattern = xlwt.Pattern.SOLID_PATTERN
cell_pattern.pattern_fore_colour = 9
cell_font = xlwt.Font()
border = xlwt.Borders()
border.right = xlwt.Borders.THIN
border.left = xlwt.Borders.THIN
border.top = xlwt.Borders.THIN
border.bottom = xlwt.Borders.THIN

cell_style = xlwt.XFStyle()
cell_style.pattern = cell_pattern
cell_style.borders = border

# side

side_style = deepcopy(cell_style)
side_style.pattern.pattern_fore_colour = 22
side_style.alignment.wrap = 1

# delta style 
delta_style = deepcopy(cell_style)
delta_style.num_format_str = '0%'

# value style
value_style = deepcopy(cell_style)
value_style.num_format_str = 'General'

# noticeable style
notice_style = deepcopy(delta_style)
notice_style.pattern.pattern_fore_colour = 26

In [27]:
#Check trios
from itertools import combinations

# Create new workbook object
book = xlwt.Workbook(encoding='utf-8')
sheet1 = book.add_sheet('eae')

max_pair_width = 0
max_npc_width = 0
min_head_width = 0

Corr = More.corr()

header_list = ['Obeserved Vars', 'Pairs', 'Baseline Corr', 'P_value (2t)', 'Naïve Partial Corr', 'Naïve Partial Corr', 'Delta', 'P_value (2t)']
row_index = 0
i = 0
for trio in CheckList1:
    i+=1
    print(i)
    print(trio)

    for idx, head in enumerate(header_list):
        sheet1.write(row_index, idx, head, header_style)
        
        current_len = get_width(len(head))
        if current_len > min_head_width:
            min_head_width = current_len
    row_index += 1
    
    trio_label = ','.join(trio)
    sheet1.write_merge(row_index, row_index + 2, 0, 0, trio_label, style = side_style)
    
    #The checked sub-df
    Trios = More[trio]
    Trios = np.hstack((np.ones((Trios.shape[0],1)), Trios))
    partial_corr_array = Trios
    pair = partial_corr(partial_corr_array, weights)

    conditianl_cov_for_all = pair[0]
    conditianl_pvalue_for_all = pair[1]
    print(conditianl_cov_for_all)
    
    corrdf = allVSall_to_df(conditianl_cov_for_all,trio)
    pvaluedf = allVSall_to_df(conditianl_pvalue_for_all,trio)
    
    
    for idx, pair in enumerate(combinations(trio, 2)):
        pair_label = ','.join(pair)
        sheet1.write(row_index + idx, 1, pair_label, cell_style)
        
        current_len = get_width(len(pair_label))
        if current_len > max_pair_width:
            max_pair_width = current_len
        
#         baseline_corr = Corr.loc[pair[0], pair[1]]
#         baseline_pvalue = stats.pearsonr(More[pair[0]], More[pair[1]])[1]
        
        baseline_corr = wpearson(More[pair[0]],More[pair[1]],weights)
        baseline_pvalue = my_pvalue(baseline_corr, More.shape[0])
    
        sheet1.write(row_index + idx, 2, baseline_corr, value_style)
        sheet1.write(row_index + idx, 3, baseline_pvalue, value_style)
        
        npc_label = "{},{}|{}".format(*pair, *set(trio).difference(pair))
        npc = corrdf.loc[pair[0], pair[1]]
        pvalue = pvaluedf.loc[pair[0], pair[1]]
        delta = (baseline_corr - npc) / baseline_corr
        
        current_len = get_width(len(npc_label))
        if current_len > max_npc_width:
            max_npc_width = current_len
        
        sheet1.write(row_index + idx, 4, npc_label, cell_style)
        sheet1.write(row_index + idx, 5, npc, value_style)
        
        if delta > 0.2:
            sheet1.write(row_index + idx, 6, delta, notice_style)
        else:
            sheet1.write(row_index + idx, 6, delta, delta_style)
            
        sheet1.write(row_index + idx, 7, pvalue, value_style)
        
    row_index += 3

for i in range(7):
    sheet1.col(i).width = min_head_width
sheet1.col(1).width = max_pair_width
sheet1.col(4).width = max_npc_width

1
['All_EAE', 'cann_init_age_1', 'maud5']
[[ 1.      0.4568  0.2204 -0.0057]
 [ 0.4568  1.      0.1277  0.0375]
 [ 0.2204  0.1277  1.      0.2331]
 [-0.0057  0.0375  0.2331  1.    ]]
2
['All_EAE', 'nico_init_age_1', 'maud5']
[[1.     0.4666 0.2716 0.0152]
 [0.4666 1.     0.1356 0.0546]
 [0.2716 0.1356 1.     0.1028]
 [0.0152 0.0546 0.1028 1.    ]]
3
['All_EAE', 'alc_init_age_1', 'maud5']
[[ 1.      0.3478  0.7151 -0.0075]
 [ 0.3478  1.      0.0917  0.0643]
 [ 0.7151  0.0917  1.      0.0539]
 [-0.0075  0.0643  0.0539  1.    ]]
4
['All_EAE', 'not_alc_nico_cann_init_age_1', 'maud5']
[[1.     0.3883 0.1195 0.0266]
 [0.3883 1.     0.1213 0.0498]
 [0.1195 0.1213 1.     0.1521]
 [0.0266 0.0498 0.1521 1.    ]]
5
['All_EAE', 'ppyaud5', 'maud5']
[[1.     0.4315 0.1661 0.0264]
 [0.4315 1.     0.1242 0.0532]
 [0.1661 0.1242 1.     0.1233]
 [0.0264 0.0532 0.1233 1.    ]]
6
['All_EAE', 'pnicdep5', 'maud5']
[[1.     0.4177 0.1416 0.029 ]
 [0.4177 1.     0.1346 0.0526]
 [0.1416 0.1346 1.     0.1181]
 

In [154]:
book.save(f"{EAE_event}_maud.xls")


### Part 3 - Extra exploration

In [151]:
import itertools

def weighted_corr_mat(colnames, weights = weights):
    ind = {}
    for index, value in enumerate(colnames):
        ind[value] = index
    
    pairs = itertools.combinations(colnames, 2)
    pairs = list(pairs)
    weight_corr = np.eye(len(colnames))
    
    for pair in pairs:
        a = pair[0]
        b = pair[1]
        
        corr = wpearson(More[a], More[b], weights)
        weight_corr[ind[a]][ind[b]] = corr
        weight_corr[ind[b]][ind[a]] = corr

    return(weight_corr, ind)
#     return allVSall_to_df(weight_corr, trio=colnames)
    
weighted_corr_mat(['any_sud_py','cann_init_age_1','personality_dis2','nethrace_2','nsex_1','eae_violence_1'])

(array([[ 1.    ,  0.1292,  0.1441,  0.0057,  0.0094,  0.0652],
        [ 0.1292,  1.    ,  0.1911, -0.0134,  0.1206,  0.1298],
        [ 0.1441,  0.1911,  1.    ,  0.0079,  0.0185,  0.2349],
        [ 0.0057, -0.0134,  0.0079,  1.    , -0.0231,  0.0453],
        [ 0.0094,  0.1206,  0.0185, -0.0231,  1.    ,  0.0012],
        [ 0.0652,  0.1298,  0.2349,  0.0453,  0.0012,  1.    ]]),
 {'any_sud_py': 0,
  'cann_init_age_1': 1,
  'personality_dis2': 2,
  'nethrace_2': 3,
  'nsex_1': 4,
  'eae_violence_1': 5})

In [152]:
def deltacorr(var, row_name, col_name, color='#ffffcc'):

    attr = 'background-color: {}'.format(color)
    delta = abs(var - weighted_corr_mat[row_name][col_name]) / abs(weighted_corr_mat[row_name][col_name])
    if delta > 0.2: return color 
    else: return ''
        
# deltacorr(corrdf['any_sud_py'])
# type(corrdf['any_sud_py'])


In [153]:
# Take from DB (wave3_NS4) only VariablesForUse columns
trio = ['maud5','cann_init_age_1',
       'personality_dis2','nethrace_2','nsex_1','eae_violence_1']
# 'pnicdep5','any_anx_py','any_manic_py','eatd_py',
#  'ppyaud5','nico_init_age_1','alc_init_age_1',,'any_disphor_py','any_disphor_ppy'
Trios = More[trio]
Trios = np.hstack((np.ones((Trios.shape[0],1)), Trios))
partial_corr_array = Trios
pair = partial_corr(partial_corr_array, weights)

conditianl_cov_for_all = pair[0]
conditianl_pvalue_for_all = pair[1]

corrdf = allVSall_to_df(conditianl_cov_for_all,trio)
pvaluedf = allVSall_to_df(conditianl_pvalue_for_all,trio)
# weighted_corr = weighted_corr_mat(trio)


corrdf

,maud5,cann_init_age_1,personality_dis2,nethrace_2,nsex_1,eae_violence_1
Var,,,,,,
maud5,1.0000,0.2091,0.1148,0.0482,0.0328,0.0221
cann_init_age_1,0.2091,1.0000,0.1369,-0.0264,0.1102,0.0841
personality_dis2,0.1148,0.1369,1.0000,-0.0051,-0.0052,0.2118
nethrace_2,0.0482,-0.0264,-0.0051,1.0000,-0.0225,0.0448
nsex_1,0.0328,0.1102,-0.0052,-0.0225,1.0000,-0.0136
eae_violence_1,0.0221,0.0841,0.2118,0.0448,-0.0136,1.0000


In [279]:
DB['cann_py'].value_counts()

     25122
0     7486
1     3701
Name: cann_py, dtype: int64